# sprint　機械学習スクラッチ入門

In [74]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDRegressor

In [75]:
# データのロード
iris = load_iris()
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = pd.DataFrame(data=iris.target, columns=['species'])
df = pd.concat([X, y], axis=1)
# 抜き出し
df_2ex =df.query('species == 1 | species == 2')[["sepal length (cm)", "petal length (cm)", 'species']]
X = df_2ex[["sepal length (cm)", "petal length (cm)",]]
y = df_2ex[["species"]]
# ndarray変換
X = X.values
# yに関して一次元化（グラフ化で必要）
y = np.ravel(y.values)

# 【問題1】train_test_splitのスクラッチ
スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。以下の雛形をベースとして関数を完成させてください。

sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation

なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

In [76]:
def scratch_train_test_split(X, y, train_size=0.8,):
    """
    検証用データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      学習データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    # 分割前のn_samplesの入手
    n_before = X.shape[0]
    # train_sizeを元に分割後のtrainのn_samplesを入手
    n_train = round(n_before * train_size)
    # 分割後のn_samplesの数だけランダムにインデックスを抽出
    index_train = random.sample(range(n_before), k=n_train)
    # ランダムインデックスを用いて分割
    X_train = X[index_train, :]
    y_train = y[index_train]
    X_test = np.delete(X, [index_train], 0)
    y_test = np.delete(y, [index_train], 0)

    return X_train, X_test, y_train, y_test

In [77]:
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)
print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

X_train (80, 2)
X_test (20, 2)
y_train (80,)
y_test (20,)


分類は3種類の手法をスクラッチします。

* ロジスティック回帰
* SVM
* 決定木  

ロジスティック回帰はscikit-learnにおいてLogisticRegressionクラスとSGDClassifierクラスの2種類から使用できます。ここでは勾配降下法を用いて計算するSGDClassifierクラスを利用してください。引数でloss="log"とすることでロジスティック回帰の計算になります。

sklearn.linear_model.SGDClassifier — scikit-learn 0.21.3 documentation
sklearn.svm.SVC — scikit-learn 0.21.3 documentation
sklearn.tree.DecisionTreeClassifier — scikit-learn 0.21.3 documentation
データセットは3種類用意します。

1つ目は事前学習期間同様にirisデータセットです。

sklearn.datasets.load_iris — scikit-learn 0.20.2 documentation

2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類全て使います。

virgicolorとvirginica
残り2つは特徴量が2つのデータセットを人工的に用意します。以下のコードで説明変数X,目的変数yが作成可能です。「シンプルデータセット1」「シンプルデータセット2」とします。特徴量が2つであるため可視化が容易です。

# 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

* irisデータセットは上記でロードずみ

In [50]:
# シンプルデータセット１
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X1 = X[random_index]
y1 = y[random_index]

In [51]:
# シンプルデータセット２
X2 = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y2 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [66]:
# irisデータセット
sgd0 = SGDClassifier(loss='log')
sgd0.fit(X, y)

svc0 = SVC()
svc0.fit(X, y)

dtc0 = DecisionTreeClassifier()
dtc0.fit(X, y)

print("SGDClassifier\n", sgd0.predict(X))
print("\nSVC\n", svc0.predict(X))
print("\nDecisionTreeClassifier\n", dtc0.predict(X))

SGDClassifier
 [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [67]:
# シンプルデータセット1
sgd1 = SGDClassifier(loss='log')
sgd1.fit(X1, y1)

svc1 = SVC()
svc1.fit(X1, y1)

dtc1 = DecisionTreeClassifier()
dtc1.fit(X1, y1)

print("SGDClassifier\n", sgd1.predict(X1))
print("\nSVC\n", svc1.predict(X1))
print("\nDecisionTreeClassifier\n", dtc1.predict(X1))

SGDClassifier
 [-1  1  1  1  1  1 -1 -1 -1 -1 -1  1  1 -1  1  1  1 -1 -1  1  1  1 -1 -1
  1 -1  1 -1  1 -1  1  1  1 -1  1  1  1  1 -1 -1  1 -1 -1  1 -1 -1  1  1
  1 -1 -1  1  1  1  1 -1  1  1  1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1  1
 -1  1  1  1 -1 -1  1  1  1  1  1 -1  1 -1 -1  1  1 -1 -1  1  1 -1  1  1
  1 -1  1 -1 -1 -1  1 -1  1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1 -1  1  1  1
  1  1 -1  1  1 -1 -1 -1 -1  1  1  1  1 -1 -1  1 -1  1 -1 -1 -1  1  1  1
  1  1 -1  1  1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1  1  1  1  1 -1 -1  1 -1
 -1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1  1 -1
 -1 -1  1 -1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1  1 -1 -1  1  1 -1 -1  1
 -1 -1  1  1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1
  1 -1  1 -1 -1  1 -1 -1  1 -1  1  1 -1  1  1  1 -1  1 -1  1 -1 -1 -1  1
  1  1  1  1 -1  1 -1 -1  1  1  1  1  1 -1  1 -1 -1  1  1  1  1 -1  1  1
 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1  1 -1 -1  1  1 -1 -1  1  1  1  1
  1  1 -1 -1  1 -1 -1  1  1  1 -1  1

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [68]:
# シンプルデータセット2
sgd2 = SGDClassifier(loss='log')
sgd2.fit(X2, y2)

svc2 = SVC()
svc2.fit(X2, y2)

dtc2 = DecisionTreeClassifier()
dtc2.fit(X2, y2)

print("SGDClassifier\n", sgd2.predict(X2))
print("\nSVC\n", svc2.predict(X2))
print("\nDecisionTreeClassifier\n", dtc2.predict(X2))

SGDClassifier
 [0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1
 1 1 1]

SVC
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 1 1 1]

DecisionTreeClassifier
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# 回帰問題
回帰は1種類をスクラッチします。

* 線形回帰  

線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用してください。

sklearn.linear_model.SGDRegressor — scikit-learn 0.21.3 documentation

データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。

House Prices: Advanced Regression Techniques

train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

# 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [79]:
# House Pricesデータセットロード
# データセットの準備
df_origin = pd.read_csv("../example_input/house_prices/train.csv")
df = df_origin[["GrLivArea", "YearBuilt", "SalePrice"]]
X_house = df[["GrLivArea", "YearBuilt"]]
y_house = df[["SalePrice"]]

In [81]:
display(X_house.head())
display(y_house.head())

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [83]:
sgdr_house = SGDRegressor()
sgdr_house.fit(X_house, y_house)
sgdr_house.predict(X_house)

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2.27834515e+15, 1.83469832e+15, 2.35158068e+15, ...,
       2.87144472e+15, 1.64789545e+15, 1.82548123e+15])